In [312]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
from pylab import rcParams
rcParams['figure.figsize'] = 8, 5

In [313]:
df = pd.read_csv('train_tutors.csv')
df_test = pd.read_csv('test_tutors.csv')

In [315]:
data = df.values
data_test = df_test.values

In [316]:
data.shape

(10000, 13)

In [317]:
data_test.shape

(10000, 12)

In [318]:
label = data[:, -1]
data = data[:, 1:12]
data_test = data_test[:, 1:12]

In [319]:
print(data.shape)
print(data_test.shape)

(10000, 11)
(10000, 11)


In [328]:
random.seed(42)

def get_bootstrap(data, labels, N):
    n_samples = data.shape[0]
    bootstrap = []
    
    for i in range(N):
        b_data = np.zeros(data.shape)
        b_labels = np.zeros(labels.shape)
        
        for j in range(n_samples):
            sample_index = random.randint(0, n_samples-1)
            b_data[j] = data[sample_index]
            b_labels[j] = labels[sample_index]
        bootstrap.append((b_data, b_labels))
        
    return bootstrap

In [329]:
def get_subsample(len_sample):
    # будем сохранять не сами признаки, а их индексы
    sample_indexes = [i for i in range(len_sample)]
    
    len_subsample = int(np.sqrt(len_sample))
    subsample = []
    
    random.shuffle(sample_indexes)
    for _ in range(len_subsample):
        subsample.append(sample_indexes.pop())
        
    return subsample

In [330]:
# Реализуем класс узла

class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

In [331]:
# И класс терминального узла (листа)

class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
    
    def predict(self):
        # подсчет вероятности объектов класса 1
        return list(self.labels).count(1)/len(self.labels)

In [332]:
# Расчет энтропии Шеннона

def criterion(labels):
    #  подсчет количества объектов разных классов
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1
    
    #  расчет критерия
    entrp = 0
    delta = 1e-5
    for label in classes:
        p = classes[label] / len(labels)
        entrp += -p * np.log2((p + delta))
        
    return entrp

In [334]:
# Расчет качества

def quality(left_labels, right_labels, current_criterion):

    # доля выбоки, ушедшая в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
    return current_criterion - p * criterion(left_labels) - (1 - p) * criterion(right_labels)

In [335]:
# Разбиение датасета в узле

def split(data, labels, index, t):
    
    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)
        
    true_data = data[left]
    false_data = data[right]
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

In [336]:
# Нахождение наилучшего разбиения

def find_best_split(data, labels):
    
    #  обозначим минимальное количество объектов в узле
    min_leaf = 1

    current_criterion = criterion(labels)

    best_quality = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    # выбор индекса из подвыборки длиной sqrt(n_features)
    subsample = get_subsample(n_features)
    
    for index in subsample:
        # будем проверять только уникальные значения признака, исключая повторения
        t_values = np.unique([row[index] for row in data])
        
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_leaf or len(false_data) < min_leaf:
                continue
            
            current_quality = quality(true_labels, false_labels, current_criterion)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_quality > best_quality:
                best_quality, best_t, best_index = current_quality, t, index

    return best_quality, best_t, best_index

In [337]:
# Построение дерева с помощью рекурсивной функции

def build_tree(data, labels, max_depth=np.inf, min_q=0.0, n_elem=1):

    quality, t, index = find_best_split(data, labels)

    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    if quality <= min_q or max_depth <= 0 or labels.size <= n_elem:
        return Leaf(data, labels)

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)

    max_depth -= 1
    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_labels, max_depth, min_q, n_elem)
    false_branch = build_tree(false_data, false_labels, max_depth, min_q, n_elem)

    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

In [338]:
def random_forest(data, labels, n_trees, max_depth=np.inf, min_q=0.0, n_elem=1):
    forest = []
    bootstrap = get_bootstrap(data, labels, n_trees)
    
    for b_data, b_labels in bootstrap:
        forest.append(build_tree(b_data, b_labels, max_depth, min_q, n_elem))
        
    return forest

In [339]:
# Функция классификации отдельного объекта

def classify_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)

In [340]:
# функция формирования предсказания по выборке на одном дереве

def predict(data, tree):
    
    classes = []
    for obj in data:
        prediction = classify_object(obj, tree)
        classes.append(prediction)
    return classes

In [341]:
# предсказание голосованием деревьев

def tree_vote(forest, data):

    # добавим предсказания всех деревьев в список
    predictions = []
    
    for tree in forest:
        predictions.append(predict(data, tree))
    
    # сформируем список с предсказаниями для каждого объекта
    predictions_per_object = list(zip(*predictions))
    
    # выберем в качестве итогового предсказания для каждого объекта то,
    # за которое проголосовало большинство деревьев
    voted_predictions = []
    for obj in predictions_per_object:
        voted = np.mean(obj)
        voted_predictions.append(voted)
        
    return voted_predictions

Строим модель на обычных данных

In [361]:
train_data, test_data, train_labels, test_labels = train_test_split(data, label, test_size = 0.3, random_state = 1)

In [455]:
n_trees = [1000]
max_depth = [11]
min_q = 0.00
n_elem = 3

In [456]:
%%time
my_forests2 = []
for md in max_depth:
    for t in n_trees:
        my_forest = random_forest(train_data, train_labels, t, md, min_q, n_elem)
        my_forests2.append((my_forest, t, md))

Wall time: 20min 59s


Посчитаем тестовый датасет:

In [459]:
# Получим ответы для ответов 
test_prediction = tree_vote(my_forests2[0][0], data_test)

In [460]:
df_test['choose'] = test_prediction

In [461]:
df_test[['Id', 'choose']].to_csv('result_log2_1000_2.csv', index=False)